In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, regexp_replace

spark = SparkSession.builder.appName("Clustering Analysis").getOrCreate()

df = spark.read.csv("../data/output_books.csv", header=True, inferSchema=True)

df_cleaned = df.withColumn("price", regexp_replace(col("price"), "[^0-9.]", "")) \
               .withColumn("price", col("price").cast("float"))

df_cleaned = df_cleaned.filter(df_cleaned["price"].isNotNull())

assembler = VectorAssembler(inputCols=["price"], outputCol="features")
df_kmeans = assembler.transform(df_cleaned)

kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(df_kmeans)

clusters = model.transform(df_kmeans)
clusters.select("title", "price", "features", "prediction").show(truncate=False)

spark.stop()


24/10/09 18:32:42 ERROR Instrumentation: java.lang.IllegalArgumentException: requirement failed: No samples available from MapPartitionsRDD[68] at map at KMeans.scala:223
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.mllib.clustering.KMeans.initKMeansParallel(KMeans.scala:369)
	at org.apache.spark.mllib.clustering.KMeans.runAlgorithmWithWeight(KMeans.scala:257)
	at org.apache.spark.mllib.clustering.KMeans.runWithWeight(KMeans.scala:231)
	at org.apache.spark.ml.clustering.KMeans.$anonfun$fit$1(KMeans.scala:353)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.clustering.KMeans.fit(KMeans.scala:328)
	at org.apache.spark.ml.clustering.KMeans.fit(KMeans.scala:271)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.ref

IllegalArgumentException: requirement failed: No samples available from MapPartitionsRDD[68] at map at KMeans.scala:223